<a href="https://colab.research.google.com/github/lanyu0322/phd_semester1/blob/master/park_area_within500ft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# -- install geopandas
!pip install geopandas
!pip install gmaps

import geopandas as gp
import pyproj
import os

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 14.7MB 35.3MB/s 
     |████████████████████████████████| 10.4MB 36.1MB/s 
     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076096 sha256=92206682b608b51080522b221d15278be85b0a857d5b344dbda9a09237cf844c
  Stored in directory: /root/.cache/pip/wheels/33/eb/41/5302168c36fc8ce8f48143bb58fde1015d5bc0c62bc3ddbb38
Successfully built gmaps


In [0]:
# -- read in the counts

camera_fname = os.path.join("drive", "My Drive", "lwir", "data", "nycdot", "cams_ft_wd.feather")
camera = pd.read_feather(camera_fname)
fname = os.path.join("drive", "My Drive", "lwir", "data", "nycdot", "avg_weekend_ts.feather")

# -- extract just the counts
camarr = camera.drop(columns=["cam_id", "lat", "lon"]).values
cam_len = len(camera)

In [0]:
# -- define helper function for integrating within a circle
def integrate_geodata(geo, vals, lat, lon, rad):
  
  # -- convert lat/lon to NY State Plane
  ll_nyspd = pyproj.Proj("epsg:2263", preserve_units=True)(lon, lat)
  
  # -- define a circle
  circ = Point(ll_nyspd[0], ll_nyspd[1]).buffer(rad)
  
  # -- calculate the intersection of the circle with the geometry
  inter = geo.intersection(circ)
  
  # -- determine the overlap fraction
  frac = inter.area / geo.area
  return (frac * vals).sum() 

In [8]:
# for each camera:
park_fname = os.path.join("drive", "My Drive", "lwir", "data", "parks", "properties", "geo_export_5b605687-0f81-4fe4-9fb7-ec27cd43ab5f.shp")
park = gp.read_file(park_fname)
park_data = park[park["borough"] == "M"]

print("init park epsg: ", park_data.geometry.crs)


def convert_to_nyc(input_data):
  input_data.geometry = input_data.geometry.to_crs(epsg=2263)

convert_to_nyc(park_data)
print("converted park epsg: ", park_data.geometry.crs)

print(park_data.geometry.size)
old_geometry = park_data.geometry

#set a new dataframe
#if park_data.geometry.iloc[i].is_invalid, I change this polygon to a point(0,0)

new_geometry = []
for i in range(len(park_data.geometry)):
  geo = park_data.geometry.iloc[i]
  if (not park_data.geometry.iloc[i].is_valid):
    geo = Point(0,0)
  new_geometry.append(geo)


df = pd.DataFrame(park_data)
park_data_new = gp.GeoDataFrame(df, geometry=new_geometry)

# -- get the parks area within a radius 
#park_area = np.array([integrate_geodata(park_data.geometry, park_data.area, camera.lat[i], camera.lon[i], 500.) for i in range(cam_len)])

park_area = np.array([integrate_geodata(park_data_new.geometry, park_data_new.area, camera.lat[i], camera.lon[i], 500.) for i in range(cam_len)])

init park epsg:  epsg:4326


/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[geo_column_name] = level


converted park epsg:  epsg:2263
384


In [10]:
print(convert_to_nyc(park_data))

None


/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[geo_column_name] = level


In [0]:
park_area = pd.DataFrame(park_area)
park_area.to_csv('parkarea.csv', index=False)
!cp parkarea.csv "drive/My Drive/lwir/data/4_pop_fit"

# get top4 and least4 camid

In [0]:
park_area = pd.DataFrame(park_area)
park_area.columns = ["park_area", "camid"]
park_area["camid"] = camera["cam_id"]
park_area[park_area["park_area"] == 0].iloc[0:5] # 421, 514, 176, 163, 169
park_area.sort_values(by=["park_area"], ascending=False).iloc[0:5] #524, 968, 969, 291, 445
bind   = [421, 514, 176, 163, 169]
nopark = camera[camera.cam_id.isin(bind)]
cols = nopark.columns[3:99]
avg_nopark = nopark[cols].sum(axis = 0)/5
ind = [524, 968, 969, 291, 445]
max_park = camera[camera.cam_id.isin(ind)]
avg_park = max_park[cols].sum(axis = 0)/5

In [0]:
avg_nopark = pd.DataFrame(avg_nopark)
avg_nopark.to_csv('avg_nopark.csv', index=False)
!cp avg_nopark.csv "drive/My Drive/lwir/data/4_pop_fit"

In [0]:
avg_park = pd.DataFrame(avg_park)
avg_park.to_csv('avg_park.csv', index=False)
!cp avg_park.csv "drive/My Drive/lwir/data/4_pop_fit"